In [1]:
from notebook_log_loading import load_sensor_log_file

sensor_log,first_timestamp=load_sensor_log_file("../to_study/senw_imu.txt")

In [5]:
import sys, os
parent_path = os.path.normpath(os.path.abspath(os.path.join(os.getcwd(), os.path.pardir)))
#print(path2add)
sys.path.append(parent_path)
sys.path.append(os.path.normpath(os.path.join(parent_path,"gyrii")))
sys.path.append(os.path.normpath(os.path.join(parent_path,"gyrii","underpinnings")))

from gyrii.Gyrus import ThreadedGyrus
from gyrii.underpinnings.NNCalibration2 import StatePredictorWithPolicy

def toeuler(q):
    w=q[3]
    i=q[0]
    j=q[1]
    k=q[2]
    s=1
    
    cpch=1-2*s*(j*j+k*k)
    cpsh=2*s*(i*j+k*w)
    sp=2*s*(i*k-j*w)
    sbcp=2*s*(j*k+i*w)    
    cbcp=1-2*s*(i*i+j*j)
    
    conv=360/(2*np.pi)
    #heading=360*np.arctan2(cpsh,cpch)/(2*3.14)
    #pitch=-360*np.arcsin(sp)/(2*3.14)
    #bank=360*np.arctan2(sbcp,cbcp)/(2*3.15)
    heading=conv*np.arctan2( 2*(w*i+j*k),1-2*(i*i+j*j))
    pitch=conv*np.arcsin(np.clip(2*(w*j-k*i),-1,1))    
    bank=conv*np.arctan2( 2*(w*k+i*j),1-2*(j*j+k*k))
    return [heading,pitch,bank]

class VisualMotionCalibGyrus(ThreadedGyrus):
    def __init__(self,broker,load_file=None):

        self.motionpredictor=nn.Sequential(nn.Linear(4,4),
                              nn.Tanh(),
                              nn.Linear(4,2))

        self.policy=nn.Sequential(nn.Linear(4,4),
                                  nn.Tanh(),
                                  nn.Linear(4,2),
                                  nn.Tanh())
        
        self.last_gyro=np.zeros(3) #last gyroscope reading
        self.last_accel=np.zeros(3) #last accelerometer reading
        self.next_motor_end_time=0
        self.last_yaw=0
        
        
    def get_keys(self):
        return []
    
    def get_name(self):
        return "VisualMotionCalibGyrus"
    
    def holding_still(self,timestamp): #return true if I think I'm holding still
        if np.linalg.norm(self.last_gyro)>0.03:
            return False
        if np.linalg.norm(self.last_accel)>0.15:
            return False
        if time.time()<self.next_motor_update_time:
            return False
        return True
    
    def get_unique_tagged_objects(self,tagged_objects):
        ret={}
        label_list=[ x["label"] for x in tagged_objects]
        for key in self.objects_to_watch:
            if label_list.count(key)>1 or label_list.count(key)==0:
                continue
            elem=next(filter(lambda x: x["label"]==key,tagged_objects))
            ret[key]=elem
        return ret

        
    def read_message(self,message):
        if 'packets' in message: #rotation etc
            packet=entry['packets'][-1]
            self.last_accel=packet["acceleration"]                
            self.last_gyro=packet["gyroscope"]
            euler_angles=toeuler(packet["rotation_vector"])
            self.last_yaw=euler_angles[2]
            #TODO I need to align vectors here
            
        if 'detections' in message:
            dets=get_unique_tagged_objects(message["detections"],[32,11])
            ...
            


